In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
def describe_dataset(dataset):
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-4]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [5]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features_3.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
Size: 10


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [6]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_3.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [7]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    
    print('# all matches: {}'.format(len(features_csv_files)))
    features_all_csv = features_csv_files[0:len(features_csv_files)]
    df_all_data = pd.concat(pool.map(construct_data_set, features_all_csv))

    print('All-data-set shape: {}'.format(df_all_data.shape))
    
    describe_dataset(df_all_data)

# all matches: 298
All-data-set shape: (679382, 337)
Pitch index counts
------------------
0	: 209737
1	: 19797
2	: 47201
3	: 18229
4	: 57806
5	: 169396
6	: 58463
7	: 21156
8	: 52785
9	: 24812


## Exporitng 

In [8]:
    df_all_data.to_csv('../data/general/{dataset}/data_{dataset}.csv'.format(dataset='all'), index=False)

In [9]:
    df_all_data[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsAvgSpeed,teamsMinSpeed,teamMaxSpeed,teamsMaxSprint,teamsDBSCANx,...,awayCenterForwardMinSpeedX,awayCenterForwardMinSpeedY,awayCenterForwardMinSpeed,awayCenterForwardMaxSpeedX,awayCenterForwardMaxSpeedY,awayCenterForwardMaxSpeed,pitch_index,xpos,ypos,ball_owner
1,65.750000,38.96875,2.470703,1.0,-1.0,3.423828,1.219727,5.750000,5.750000,72.68750,...,48.000000,52.78125,1.870117,48.000000,52.78125,1.870117,5.0,67.687500,40.09375,1.0
2,72.687500,44.18750,1.120117,1.0,1.0,1.157227,0.170044,1.599609,0.000000,74.12500,...,42.531250,47.34375,1.730469,42.531250,47.34375,1.730469,0.0,0.000000,0.00000,0.0
3,65.125000,40.87500,1.509766,-1.0,-1.0,2.576172,1.259766,4.410156,4.410156,74.50000,...,53.531250,56.50000,2.179688,68.125000,34.31250,2.429688,5.0,43.968750,40.34375,1.0
4,32.750000,40.12500,2.900391,1.0,-1.0,2.193359,0.899902,3.839844,3.839844,24.00000,...,18.046875,48.56250,1.349609,18.046875,48.56250,1.349609,2.0,1.389648,42.87500,1.0
5,69.312500,40.90625,2.080078,-1.0,1.0,1.335938,0.729980,1.530273,0.000000,77.43750,...,57.625000,48.87500,1.530273,57.625000,48.87500,1.530273,0.0,0.000000,0.00000,0.0
6,27.156250,31.75000,0.879883,1.0,-1.0,1.663086,0.189941,3.279297,3.279297,18.78125,...,12.976562,28.43750,1.940430,12.976562,28.43750,1.940430,2.0,6.738281,36.40625,1.0
7,44.250000,35.59375,2.480469,1.0,-1.0,1.815430,0.379883,3.130859,3.130859,46.81250,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,5.0,53.843750,24.50000,1.0
8,23.421875,51.31250,1.080078,1.0,1.0,1.600586,0.239990,3.330078,3.330078,19.06250,...,17.140625,34.00000,1.940430,17.140625,34.00000,1.940430,2.0,20.375000,46.28125,1.0
9,50.937500,29.37500,1.299805,-1.0,-1.0,1.746094,0.360107,2.730469,0.000000,44.65625,...,46.750000,28.87500,2.460938,46.750000,28.87500,2.460938,8.0,100.937500,30.15625,1.0
